In [ ]:
import numpy as np
import fastpathplanning as fpp
from uav_visualization import Village

# initializes 3d environment (run this notebook to the end, and
# then go to the link below to see the animation of the uav flight)
village = Village()

In [ ]:
# numerical parameters
village_height = 5 
village_side = 14
building_every = 5
assert (village_side + 1) % building_every == 0
p_init = np.zeros(3) # initial point
p_term = np.array([village_side, village_side, 0]) # terminal point
r_uav = .1 # uav radius

In [ ]:
# helper functions for construction of village

def direction():
    I = np.eye(2)
    directions = np.vstack((I, -I))
    return directions[np.random.randint(0, 4)]

def walk(m):
    d1 = direction()
    starts = [np.zeros(2)]
    ends = [d1]
    blocks = [np.zeros(2), d1]
    for i in range(m):
        d2 = direction()
        blocks.append(blocks[-1] + d2)
        if all(d2 == d1):
            ends[-1] += d1
        else:
            starts.append(ends[-1])
            ends.append(starts[-1] + d2)
            d1 = d2
    return np.array(starts), np.array(ends), np.array(blocks)

def building(village, i, j, m):
    starts, ends, blocks = walk(m)
    offset = np.array([i, j]) + .5
    starts += offset
    ends += offset
    blocks += [i, j]
    for k, (s, e) in enumerate(zip(starts, ends)):
        c = (s + e) / 2
        d = np.abs(e - s) + 1
        r = list(.5 * d - r_uav) + [village_height / 2]
        n = list(d) + [village_height]
        village.building(f'building_{i}_{j}_{k}', c, r, n)
    return blocks
        
def outer_box(i, j, x, y, r, zmin, zmax):
    L = [[i, j, zmin],
         [x + r, j, zmin],
         [i, y + r, zmin],
         [i, j, zmin]]
    U = [[x - r, j + 1, zmax],
         [i + 1, j + 1, zmax],
         [i + 1, j + 1, zmax],
         [i + 1, y - r, zmax]]
    return L, U
        
def tree(village, i, j):
    name = f'tree_{i}_{j}'
    r = .5 - r_uav # Radius of foliage.
    r_trunk = .1
    low = [i + .5, j + .5, 1]
    high = [i + .5, j + .5, village_height - .5]
    c = np.random.uniform(low, high)
    village.tree(name, c, r, r_trunk)
    L, U = outer_box(i, j, c[0], c[1], r_trunk + r_uav, 0, c[2] - .5) # Trunk.
    L.append([i, j, c[2] + .5]) # Above the tree.
    U.append([i + 1, j + 1, village_height]) # Above the tree.
    return L, U
        
def bush(village, i, j):
    name = f'bush_{i}_{j}'
    r = np.random.uniform(low=.1, high=.35) # Radius.
    h = 4 * r # Height.
    c = np.array([i + .5, j + .5])
    village.bush(name, c, r, h)
    L, U = outer_box(i, j, c[0], c[1], r + r_uav, 0, h + r_uav) # Bush.
    L.append([i, j, h + r_uav]) # Above the bush.
    U.append([i + 1, j + 1, village_height]) # Above the bush.     
    return L, U

In [ ]:
village.delete()
np.random.seed(0)

# village ground
c_ground = (.7, 1, .7) # color
ground = village.ground(village_side, c_ground)

# start and ending platforms
r_platform = .15 # radius
c_platform = (1, 1, 1) # color
start_platform = village.platform('start_platform', *p_init[:2], r_platform, c_platform)
goal_platform = village.platform('goal_platform', *p_term[:2], r_platform, c_platform)

# buildings
blocks = []
for i in range(village_side):
    for j in range(village_side):
        if (i + 1) % building_every == 0 and (j + 1) % building_every == 0:
            blocks.append(building(village, i, j, 4))
blocks = [tuple(b) for b in np.vstack(blocks)]
    
# free-space boxes
L = []
U = []
for i in range(village_side):
    for j in range(village_side):
        if (i, j) not in blocks:
            if np.random.choice([0, 1]) == 0:
                Lij, Uij = tree(village, i, j)
            else:
                Lij, Uij = bush(village, i, j)
            L += Lij
            U += Uij

In [ ]:
# offline preprocessing
S = fpp.SafeSet(L, U)

# this will take a while if using Clarabel as a solver
# for the optimization of the representative points
# (for better performance consider using Mosek)

In [ ]:
# online path planning
T = village_side + 1 # traversal time
alpha = [0, 0, 0, 1] # cost weights
der_init = {1: 0, 2: 0, 3: 0} # initial conditions for derivatives
der_term = der_init # final conditions for derivatives
p = fpp.plan(S, p_init, p_term , T, alpha, der_init, der_term)

In [ ]:
def differential_flatness(p, pdd, yaw=np.pi/4):
    g = np.array([0, 0, 9.81])
    xC = np.array([np.cos(yaw), np.sin(yaw), 0])
    T = np.eye(4)
    T[:3, 3] = p
    z = g + pdd
    z /= np.linalg.norm(z)
    T[:3, 2] = z
    y = np.cross(z, xC)
    y /= np.linalg.norm(y)
    T[:3, 1] = y
    x = np.cross(y, z)
    T[:3, 0] = x
    return T

In [ ]:
from meshcat.animation import Animation, convert_frames_to_video
from meshcat.transformations import translation_matrix

# on-board animation of the flight
anim = Animation()
uav = village.uav('uav')
pdd = p.derivative().derivative()
t_samples = np.linspace(0, p.duration, 1000)
p_samples = [p(ti) for ti in t_samples]
pdd_samples = [pdd(ti) for ti in t_samples]
village.trajectory('traj', p_samples)

fps = 30
for ti, pi, pddi in zip(t_samples, p_samples, pdd_samples):
    with anim.at_frame(village, ti * fps) as frame:
        tranform = differential_flatness(pi, pddi)
        tranform_inv = np.linalg.inv(tranform)
        frame.set_transform(tranform_inv)
        frame['uav'].set_transform(tranform)
        
# constructs animation
village.set_animation(anim, play=False)

# now click the meshcat link in the top cell to see the animation
# in the top right of the meshcat window click:
# "Open Controls" -> "Animations" -> "default" -> "play"